In [9]:
from functions.plot_funcs import (
    create_cost_breakdown_sunburst,
    create_interest_rate_sensitivity_chart,
    create_amortization_chart,
    create_amortization_chart_optimized
)
from functions.calc_funcs import (
    monthly_price_calculator_scenarios,
    calculate_amortization_schedule
)

import numpy as np
import plotly.graph_objects as go
import plotly.express as px


In [10]:
# Use monthly_price_calculator_scenarios to generate realistic data
sunburst_data = monthly_price_calculator_scenarios(
    houseprice_range=[5000000],
    interest_rate_range=[0.06],
    fixed_cost_house_range=[5000],
    kwh_usage_range=[500],
    kwh_price_range=[1.5],
    markup_nok_range=[0.1],
    fixed_cost_electricity_range=[39],
    ammortisation_periods_range=[360],
    person_a_fixed_costs_range=[10000],
    person_b_fixed_costs_range=[10000],
    transaction_costs_range=[200000],
    ek_range=[1500000],
    ownership_fraq_range=[0.5]
)

# Data for interest rate sensitivity chart
loan_amount = sunburst_data['house_price'].copy() - 1500000  # Assuming ek is 1500000
amortization_periods = 360
interest_rate_range = sunburst_data['interest_rate'].copy().unique()

In [11]:
df = calculate_amortization_schedule(loan_amount, interest_rate_range, amortization_periods)

In [12]:
%%timeit
create_amortization_chart(df, 'test')

16.2 ms ± 96.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit
create_amortization_chart_optimized(df, 'test')

16.1 ms ± 129 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
def loan_calc_v1(
    loan: int | float, rate: float | np.ndarray, months: int
) -> int | np.ndarray:
    """
    Calculate the monthly payment for a given loan amount with a given interest rate over a specified number of months.

    Parameters
    ----------
    loan : float
        The loan amount.
    rate : float
        The annual interest rate.
    months : int
        The number of months over which the loan will be amortized.

    Returns
    -------
    float or np.ndarray
        The monthly payment.

    Examples
    --------
    >>> loan_calc(100000, 0.035, 360)
    448.64
    """
    return (rate / 12) * (1 / (1 - (1 + rate / 12) ** (-months))) * loan

In [33]:
def loan_calc_v2(
    loan: np.ndarray | float, rate: np.ndarray | float, months: int
) -> np.ndarray | float:
    """
    Calculate the monthly payment for given loan amounts with given interest rates over a specified number of months.
    Supports broadcasting between loan amounts and interest rates.

    Parameters
    ----------
    loan : np.ndarray or float
        The loan amount(s).
    rate : np.ndarray or float
        The annual interest rate(s).
    months : int
        The number of months over which the loan(s) will be amortized.

    Returns
    -------
    np.ndarray or float
        The monthly payment(s) for each combination of loan and rate.

    Examples
    --------
    >>> loan_calc(100000, 0.035, 360)
    448.64
    >>> loan_calc(np.array([100000, 200000]), np.array([0.035, 0.04]), 360)
    array([[448.64, 477.42],
           [897.28, 954.83]])
    >>> loan_calc(np.array([100000, 200000, 300000]), 0.035, 360)
    array([448.64, 897.28, 1345.92])
    """
    loan = np.atleast_1d(loan)[:, np.newaxis]
    rate = np.atleast_1d(rate)[np.newaxis, :]
    
    monthly_rate = rate / 12
    numerator = monthly_rate * (1 + monthly_rate) ** months
    denominator = (1 + monthly_rate) ** months - 1
    
    result = (numerator / denominator) * loan
    
    # If both inputs were scalars, return a scalar
    if result.size == 1:
        return result.item()
    
    # If one input was scalar and the other 1D, return 1D array
    if result.shape[0] == 1 or result.shape[1] == 1:
        return result.ravel()
    
    return result

In [37]:
loan_vec = np.arange(3000000, 7000000, step=25000)
interest_rates = np.arange(0.01, 0.125, 0.0025)
loan = 50000000


In [40]:
loans_v1 = np.array([loan_calc_v1(loan, rate, 360) for rate in interest_rates])
loans_v2 = loan_calc_v2(loan, interest_rates, 360)

In [44]:
np.isclose(loans_v1, loans_v2,rtol=0.01)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [45]:
%%timeit
[loan_calc_v1(loan, rate, 360) for rate in interest_rates]

23.6 μs ± 84 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [46]:
%%timeit
loan_calc_v2(loan, interest_rates, 360)

11.3 μs ± 46.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
